# Nested 10-fold cross validation on the 12-class data set from Aliper et al. 

### Uses validation set index 2 (validx1) to produce the confusion matricies in figure 3 of the paper

## requires fastai version 0.7

#### Install instructions here: https://forums.fast.ai/t/fastai-v0-7-install-issues-thread/24652

In [1]:
import matplotlib.pyplot as plt
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.imports import *
from fastai.torch_imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from random import sample
from itertools import chain
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, average_precision_score
from sklearn.metrics import matthews_corrcoef, balanced_accuracy_score, accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer, MultiLabelBinarizer
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import StratifiedKFold
import shutil
import csv
import time

#### set some variables and set your GPU if you have multiple

In [2]:
torch.cuda.set_device(1)   ### use first GPU if you have many
PATH = "data/data_alipermodel_copy/pics/" ### path to where your pictures are downloaded and the .csv files with val sets
sz = 150  ### resize images to this px by px
arch = resnext101_64  ### pre-trained network choice

## functions


In [3]:
def get_val_idx_fromfile(validx_csv):
    validx_df =pd.read_csv(validx_csv, header=None)
    return validx_df[0].tolist()
    
def get_data(sz, bs, val_idxs, label_csv): # sz: image size, bs: batch size
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_top_down, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', label_csv, val_idxs=val_idxs, 
                                        suffix='.png', 
                                        tfms=tfms, 
                                        num_workers=1,
                                        bs=bs)
    return data if sz > 300 else data.resize(150, 'tmp')

### Function to compute multiclass ROC AUC score from model predictions
def multiclass_roc_auc_score(y_test, y_probs, AVERAGE="weighted"):
    oh = OneHotEncoder(sparse=False, categories='auto')
    yt = oh.fit_transform(y_test.reshape(-1,1))
    return roc_auc_score(yt, y_probs, average=AVERAGE)

### File-based validation sets wont work for nested CV
#### Make the new subfiles with nested CV label csv and adjusted indexes

In [4]:
label_csv = f'{PATH}12cls_aliper.csv'
valididx_base = '12cls_aliper_10fold'

In [5]:
# make subset label_csv files with the different hold-out folds
df = pd.read_csv(label_csv)
tmpdf = df
df.head()

,pngpath,class,smiles
0,AntiInfectiveAgents/0,AntiInfectiveAgents,Clc1ccc(C(Cn2ccnc2)OCc2csc3c(Cl)cccc23)c(Cl)c1
1,AntiInfectiveAgents/1,AntiInfectiveAgents,CC[C@H]1OC(=O)[C@H](C)[C@@H](O[C@H]2C[C@@](C)(...
2,AntiInfectiveAgents/2,AntiInfectiveAgents,COc1cccc2c1cc([N+](=O)[O-])c1c(C(=O)O)cc3c(c12...
3,AntiInfectiveAgents/3,AntiInfectiveAgents,O=C(N[C@H](CO)[C@H](O)c1ccc([N+](=O)[O-])cc1)C...
4,AntiInfectiveAgents/4,AntiInfectiveAgents,CC[C@@H](C(=O)[C@@H](C)[C@@H](O)[C@H](C)CCc1cc...


In [6]:
vid_dict = {}
for x in range (10):
    vid_dict[x] = get_val_idx_fromfile(f'{PATH}'+valididx_base+str(x)+'.csv')

In [7]:
## loop through the validation sets, pop them from the df, write new df called 'inner#'
# then find the new positions of the remaining indexes, write new dfs called innner#_validx#
allfolds = [x for x in list(range(10))]
for x in allfolds:
    print(x)
    tmpdf = df.drop(vid_dict[x])
    tmpdf.to_csv(f'{PATH}12cls_nCV_inner'+str(x)+ '.csv', index=False)
    tmpfolds = [i for i in list(range(10))]
    tmpfolds.remove(x)
    for y in tmpfolds:
        tmpnewvalidx = [tmpdf.index.get_loc(z) for z in vid_dict[y]]
        with open(f'{PATH}12cls_nCV_inner' +str(x)+'validx'+str(y)+'.csv', 'w') as f:
            writer = csv.writer(f)
            for val in tmpnewvalidx:
                writer.writerow([val])

0
1
2
3
4
5
6
7
8
9


### 3-class, 10-fold cross validation, shuffled validation indexes

In [5]:
# make small simple dictionary of parameter values
'''
values list: 
1: learning rate
2: cycles
3: precompute T/F
4: dropout (when precompute is False)
'''
batch_sizes = [25]
learning_rates = [[1e-4,1e-3,1e-2]]
cycles = [7]
#precompute = [True, False]

dropout = [0.2, 0.4, 0.6]
noprecomp_params = [cycles, dropout, batch_sizes, learning_rates]
param_grid_list = list(itertools.product(*noprecomp_params))

In [6]:
param_grid_list

[(7, 0.2, 25, [0.0001, 0.001, 0.01]),
 (7, 0.4, 25, [0.0001, 0.001, 0.01]),
 (7, 0.6, 25, [0.0001, 0.001, 0.01])]

In [7]:
class MyFilter(object):
    def __init__(self, level):
        self.__level = level
    def filter(self, logRecord):
        return logRecord.levelno <= self.__level

In [8]:
import logging
LOG_FILENAME = '/home/jgmeyer2/fastai/courses/dl1/data/data_alipermodel_copy/pics/nestedCV_bs25_lr01.log'
logger = logging.getLogger()
fhandler = logging.FileHandler(filename=LOG_FILENAME, mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.INFO)
fhandler.addFilter(MyFilter(logging.INFO))
#logging.info('This message should go to the log file')

In [9]:
st = time.time()
### build nested cv loop
n_classes = 12
label_csv_base = f'{PATH}12cls_nCV_inner'
valididx_base = f'{PATH}12cls_nCV_inner'
label_csv = f'{PATH}12cls_aliper.csv'
outer_valididx_base = '12cls_aliper_10fold'
allfolds = [x for x in list(range(10))]
inner_means = {}
inner_stds = {}
best_params = {}
outer_acc = []
outer_roc = []
outer_mcc = []
outer_bac = []
outer_avp = []
total_models_trained = 0
for tmptest in allfolds:  ### outer loop setting each single fold as holdout
    mean_acc = []
    mean_roc = []
    mean_mcc = []
    mean_bac = []
    mean_avp = []
    std_acc = []
    std_roc = []
    std_mcc = []
    std_bac = []
    std_avp = []
    current_paramset_index = 0
    for params in param_grid_list:
        print('new parameters set= '  + str(current_paramset_index)+'params='+str(params))
        logging.info('new parameters set= '  + str(current_paramset_index)+'params='+str(params))
        # values to record per group
        acc = []
        roc = []
        mcc = []
        bac = []
        avp = []
        tmpfolds = [i for i in list(range(10))]
        tmpfolds.remove(tmptest)
        for rep in tmpfolds:
            print('inner loop= '+str(tmptest)+', paramset= '+str(current_paramset_index)+', holdout= '+str(rep))
            logging.info('inner loop= '+str(tmptest)+', paramset= '+str(current_paramset_index)+', holdout= '+str(rep))
            shutil.rmtree(f'{PATH}tmp', ignore_errors=True) # delete the tmp folder every cycle to ensure no weird behavior
            VAL_IDXs = get_val_idx_fromfile(str(valididx_base)+str(tmptest)+'validx'+str(rep)+'.csv')
            LABEL_CSV = label_csv_base+str(tmptest)+'.csv'
            data = get_data(150, 25, VAL_IDXs, LABEL_CSV)
            learn = ConvLearner.pretrained(arch, data, precompute=False, ps=params[1])
            learn.unfreeze()
            val_loss, val_acc = learn.fit(params[3], params[0], cycle_len=1, cycle_mult=2)
            ## make predictions with test-time augmentation
            log_preds, y = learn.TTA()
            probs = np.mean(np.exp(log_preds),0)
            preds = np.argmax(probs, axis=1)
            acc.append(accuracy_score(y, preds))
            #acc.append(val_acc)
            # ROC weighted
            if(rep <6):
                roc.append(multiclass_roc_auc_score(y, probs, "weighted"))
                Y = label_binarize(y, classes = [x for x in range(0,n_classes)]) 
                avp.append(average_precision_score(Y, probs, average="weighted"))# ave PR
            # balanced accuracy
            bac.append(balanced_accuracy_score(y, preds))
            # MCC
            mcc.append(matthews_corrcoef(y, preds))
            total_models_trained +=1
        #record the mean 
        mean_acc.append(round(np.mean(acc), ndigits =4))
        mean_roc.append(round(np.mean(roc), ndigits =4))
        mean_bac.append(round(np.mean(bac), ndigits =4))
        mean_mcc.append(round(np.mean(mcc), ndigits =4))
        mean_avp.append(round(np.mean(avp), ndigits =4))
        # record stdev
        std_acc.append(round(np.std(acc), ndigits =4))
        std_roc.append(round(np.std(roc), ndigits =4))
        std_bac.append(round(np.std(bac), ndigits =4))
        std_mcc.append(round(np.std(mcc), ndigits =4))
        std_avp.append(round(np.std(avp), ndigits =4))
        current_paramset_index +=1
        #print stuff
        print('params= ' + str(params) + ' mean acc = ' + str(round(np.mean(acc), ndigits =4)))
        print('std accuracy = ' + str(round(np.std(acc), ndigits =4)))
        logging.info('params= ' + str(params) + ', mean accuracy = ' + str(round(np.mean(acc), ndigits =4)))
        logging.info('std accuracy = ' + str(round(np.std(acc), ndigits =4)))                                         
    # record means for all params used in this inner loop
    inner_means[tmptest] = [mean_acc, mean_roc, mean_bac, mean_mcc, mean_avp]
    inner_stds[tmptest] = [std_acc, std_roc, std_bac, std_mcc, std_avp]
    # decide which params were best
    best_params[tmptest] = param_grid_list[np.argmax(mean_acc)]
    print('best accuracy inner loop #' +str(tmptest)+' is '+str(np.max(mean_acc)))
    print('best param set was'+str(best_params[tmptest])+', retraining on whole ')
    logging.info('best accuracy inner loop #' +str(tmptest)+' is '+str(np.max(mean_acc)))
    logging.info('best param set was'+str(best_params[tmptest])+', retraining on whole ')
    ################################################
    # retrain the whole data on those with the hold-out set as validation (original files)
    params = best_params[tmptest]
    shutil.rmtree(f'{PATH}tmp') # delete the tmp folder every cycle to ensure no weird behavior
    val_idxs = get_val_idx_fromfile(f'{PATH}'+outer_valididx_base+str(tmptest)+'.csv')
    data = get_data(sz, params[2], val_idxs, label_csv)
    learn = ConvLearner.pretrained(arch, data, precompute=False, ps= params[1])
    learn.unfreeze()
    val_loss, val_acc = learn.fit(params[3], params[0], cycle_len=1, cycle_mult=2)
    ## make predictions with test-time augmentation
    log_preds, y = learn.TTA()
    probs = np.mean(np.exp(log_preds),0)
    preds = np.argmax(probs, axis=1)
    #record stuff
    outer_acc.append(accuracy_score(y, preds))
    outer_bac.append(balanced_accuracy_score(y, preds)) # balanced accuracy
    outer_mcc.append(matthews_corrcoef(y, preds)) #MCC
    if(tmptest <6):
        outer_roc.append(multiclass_roc_auc_score(y, probs, "weighted"))
        Y = label_binarize(y, classes = [x for x in range(0,n_classes)])
        outer_avp.append(average_precision_score(Y, probs, average="weighted"))
    total_models_trained +=1
    logging.info('accuracy outer fold #' +str(tmptest)+' is '+str(accuracy_score(y, preds)))
    logging.info('####################################')
print('Done! New Skill Unlocked: nested cross validation')
print('total models trained = ' +str(total_models_trained))
et = time.time()
print('total time ='+str(et-st))

new parameters set= 0params=(7, 0.2, 25, [0.0001, 0.001, 0.01])
inner loop= 0, paramset= 0, holdout= 1


epoch      trn_loss   val_loss   accuracy                 
    0      2.601529   2.442186   0.285714  
    1      2.317027   2.233002   0.342857                 
    2      2.05442    2.219401   0.271429                 
    3      1.901867   2.335876   0.328571                 
    4      1.759638   1.987867   0.371429                 
    5      1.583125   1.855401   0.385714                 
    6      1.467764   1.788582   0.414286                 
    7      1.385764   1.883844   0.471429                 
    8      1.318453   2.106121   0.371429                 
    9      1.233257   1.592807   0.485714                 
    10     1.137146   1.714329   0.471429                 
    11     1.032681   1.656039   0.485714                 
    12     0.92658    1.662921   0.542857                  
    13     0.829646   1.662918   0.528571                  
    14     0.75905    1.675912   0.528571                  
    15     0.737602   1.866691   0.471429                  
    16  

epoch      trn_loss   val_loss   accuracy                 
    0      2.635828   3.065192   0.15942   
    1      2.406482   2.127146   0.246377                 
    2      2.111745   1.995905   0.362319                 
    3      1.994446   1.766103   0.449275                 
    4      1.802836   1.674435   0.434783                 
    5      1.643301   1.628475   0.478261                 
    6      1.483417   1.614493   0.449275                 
    7      1.409031   1.790797   0.492754                 
    8      1.33414    2.054718   0.434783                 
    9      1.255641   1.769928   0.391304                 
    10     1.156201   1.914264   0.42029                  
    11     1.04187    1.687559   0.507246                 
    12     0.932937   1.750055   0.478261                  
    13     0.825373   1.808078   0.492754                  
    14     0.769799   1.803811   0.507246                  
    15     0.773988   2.266599   0.463768                  
    16  

epoch      trn_loss   val_loss   accuracy                 
    0      2.5569     2.397155   0.202899  
    1      2.319224   1.983585   0.376812                 
    2      2.072295   2.232674   0.362319                 
    3      1.952221   1.88933    0.362319                 
    4      1.810551   2.012214   0.405797                 
    5      1.646055   1.665833   0.449275                 
    6      1.476136   1.672309   0.463768                 
    7      1.427029   2.10917    0.275362                 
    8      1.374294   1.982952   0.42029                  
    9      1.293464   1.686631   0.507246                 
    10     1.191537   1.991837   0.347826                 
    11     1.06209    1.9156     0.463768                 
    12     0.938515   1.885091   0.42029                   
    13     0.859002   1.91299    0.405797                  
    14     0.798358   1.918647   0.434783                  
    15     0.749859   2.226471   0.391304                  
    16  

epoch      trn_loss   val_loss   accuracy                 
    0      2.536317   2.203103   0.318841  
    1      2.356154   2.924937   0.26087                  
    2      2.164235   2.25595    0.391304                 
    3      2.050461   2.421364   0.304348                 
    4      1.895302   2.140127   0.347826                 
    5      1.705379   2.11455    0.376812                 
    6      1.55802    2.04629    0.391304                 
    7      1.492809   2.035017   0.434783                 
    8      1.438934   2.24286    0.405797                 
    9      1.311363   2.347175   0.391304                 
    10     1.214386   2.235904   0.391304                 
    11     1.107371   2.166594   0.405797                 
    12     1.01058    2.184746   0.405797                 
    13     0.934604   2.179276   0.376812                  
    14     0.85814    2.198012   0.376812                  
    15     0.841887   2.222307   0.405797                  
    16   

...

epoch      trn_loss   val_loss   accuracy                 
    0      2.931928   2.431317   0.212121  
    1      2.612632   2.305288   0.348485                 
    2      2.40932    2.17996    0.30303                  
    3      2.265763   1.980243   0.318182                 
    4      2.127341   1.806486   0.378788                 
    5      1.997198   1.704623   0.424242                 
    6      1.886929   1.64727    0.484849                 
    7      1.817638   2.026169   0.272727                 
    8      1.749191   1.521281   0.5                      
    9      1.698606   1.452206   0.484848                 
    10     1.601238   1.43694    0.439394                 
    11     1.500447   1.41643    0.469697                 
    12     1.428262   1.364454   0.454545                 
    13     1.357771   1.337055   0.484848                 
    14     1.270077   1.336205   0.5                      
    15     1.237789   1.54589    0.469697                 
    16     1

epoch      trn_loss   val_loss   accuracy                 
    0      2.850528   2.360337   0.215385  
    1      2.644361   2.883603   0.2                      
    2      2.413241   2.172889   0.323077                 
    3      2.278547   2.122169   0.307692                 
    4      2.093449   1.989158   0.369231                 
    5      1.952554   1.808563   0.415385                 
    6      1.795427   1.825227   0.415385                 
    7      1.744557   1.880708   0.415385                 
    8      1.712893   2.142783   0.369231                 
    9      1.63605    1.775863   0.430769                 
    10     1.567927   1.983012   0.430769                 
    11     1.485431   1.907285   0.415385                 
    12     1.351592   2.055781   0.4                      
    13     1.258038   2.016634   0.4                      
    14     1.20365    1.996707   0.430769                 
    15     1.171245   1.876077   0.415385                 
    16     1

epoch      trn_loss   val_loss   accuracy                 
    0      2.969472   2.597355   0.261538  
    1      2.721782   2.295856   0.338462                 
    2      2.462      2.117887   0.4                      
    3      2.307124   1.88448    0.415385                 
    4      2.207858   1.730517   0.507692                 
    5      2.077695   1.610749   0.476923                 
    6      1.981646   1.573686   0.446154                 
    7      1.868072   1.698372   0.476923                 
    8      1.821898   1.604862   0.553846                 
    9      1.739861   1.775416   0.507692                 
    10     1.68806    1.68068    0.584615                 
    11     1.579456   1.57488    0.553846                 
    12     1.452474   1.587217   0.538462                 
    13     1.380375   1.595572   0.507692                 
    14     1.329225   1.577777   0.507692                 
    15     1.302582   1.896665   0.4                      
    16     1

epoch      trn_loss   val_loss   accuracy                 
    0      2.921437   2.421532   0.209677  
    1      2.750006   2.318812   0.241935                 
    2      2.482565   1.919938   0.451613                 
    3      2.346443   2.298565   0.306452                 
    4      2.214744   2.116932   0.306452                 
    5      2.029248   1.859958   0.370968                 
    6      1.899192   1.841972   0.370968                 
    7      1.813961   2.163386   0.403226                 
    8      1.775531   2.091785   0.354839                 
    9      1.732332   1.983561   0.403226                 
    10     1.685477   1.817226   0.483871                 
    11     1.587583   1.806108   0.435484                 
    12     1.476627   1.867089   0.419355                 
    13     1.37812    1.816377   0.451613                 
    14     1.274141   1.822002   0.451613                 
    15     1.28379    2.368267   0.322581                 
    16     1

/home/jgmeyer2/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


In [10]:
print(outer_acc)
print(outer_bac)
print(outer_roc)
print(outer_mcc)
print(outer_avp)

[0.5342465753424658, 0.6285714285714286, 0.5942028985507246, 0.5942028985507246, 0.6231884057971014, 0.6470588235294118, 0.6060606060606061, 0.6461538461538462, 0.6923076923076923, 0.5161290322580645]
[0.36456552706552703, 0.6122488622488623, 0.44966716290245706, 0.47327019385842917, 0.6298954313660196, 0.5091938290467702, 0.4649686320809315, 0.6072991179943051, 0.6384876264555409, 0.3230010185892539]
[0.8399778362503968, 0.8838028209847906, 0.8909603293316617, 0.8416067681444871, 0.8455038696510805, 0.873776826344189]
[0.4372409980285795, 0.5548172732333606, 0.5080278284822664, 0.5089065333782516, 0.5502595494931687, 0.5718025247332853, 0.5166776261386576, 0.566424835707038, 0.6269141815872228, 0.4052562148925469]
[0.6257284917921292, 0.696711810454585, 0.7086155948164071, 0.6384455921183726, 0.6705051579283101, 0.6894130782110381]


In [11]:
print(np.mean(outer_acc))
print(np.std(outer_acc))

0.6082122207122067
0.050006838782421764
